## 🧠 LangGraph Agent with GPT-4o-mini, Tool, and Short-Term Memory
In this notebook, you'll learn how to build a simple AI agent equipped with a toold and short memory using LangGraph and OpenAI's GPT-4o-mini.

### 🔧 Step 1: Install Required Packages
We run the following cell to install the necessary packages. We’re making sure LangChain, LangGraph, and the OpenAI wrapper are on the latest version so every API we call later is available. The -U flag upgrades anything that’s already installed

In [ ]:
!pip install -U langchain langgraph langchain-openai

### 🔑 Step 2: Set Your OpenAI API Key

The agent will talk to OpenAI’s GPT-4o-mini model hosted in Azure OpenAI. Azure OpenAI uses the same models as OpenAI but allows with a different endpoint. Putting your key and endpoint in environment variables lets every downstream cell authenticate automatically without hard-coding secrets into the notebook.


### 🤖 Step 3: Import Required Libraries

Here we pull in the building blocks:

* `from langchain_openai import AzureChatOpenAI:` This brings in the AzureChatOpenAI class.
It lets you use OpenAI's chat models (like GPT-3.5 or GPT-4) in your app.

* `from langchain.agents import create_agent:` This imports a helper function to quickly build an AI agent. The agent uses the ReAct framework: it can reason about a question and then act using tools (like calling a function).
It saves you from writing a lot of setup code.

* `from langgraph.graph.message import MessagesState:` This class helps track the conversation. It keeps a record of all the messages exchanged between the user and the agent. Useful for managing context in a conversation.

* `from langchain_core.messages import HumanMessage:` This represents a message from a human. You use it to tell the agent, “This is what the user said.”

* `from langchain_core.runnables import RunnableConfig:` This is used to configure how the agent runs.

* `from langgraph.checkpoint.memory import InMemorySaver:` This sets up a memory system that stores conversation history in memory. It helps the agent remember what was said earlier in the same session.

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_agent
from langgraph.graph.message import MessagesState
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.memory import InMemorySaver

### 🧠 Step 4: Load the GPT-4o-mini Model

`AzureChatOpenAI` gives us a convenient Python object that wraps the GPT-4o-mini API. Setting `temperature=0.7` makes answers a little more creative while still fairly reliable.

In [ ]:
llm = AzureChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7
)

### 🧠 Step 5: Create a Memory Saver

`InMemorySaver()` is LangGraph’s simplest “checkpoint” store. Whenever the agent finishes a turn, it drops the final graph state here, which lets future calls pick up the same conversation thread.

In [ ]:
checkpointer = InMemorySaver()

### 🔨 Step 6: Define a Tool

This toy tool `get_animaltype` pretends to reach out to some knowledge base, but really just echoes a canned fact. The point is to show how tools can be implemented. The agent can decide when to call a tool, pass it arguments, and then weave its output back into the reply.

In [ ]:
def get_animaltype(animal: str) -> str:
    """Get type of animal simulation tool."""
    return f"{animal} have four legs!"

### 🕵️ Step 7: Define Agent

`create_agent` wires everything together using the ReAct pattern. The agent can think step-by-step, optionally call get_animaltype, remember context via the checkpoint-based memory, and then output a final answer—all in one execution.

In [ ]:
agent = create_agent(
    model=llm,
    tools=[get_animaltype],
    checkpointer=checkpointer
)

### ⚙️ Step 8: Run the Agent

The config dictionary gives LangGraph a thread_id. Any turn that uses the same ID re-hydrates the stored memory so the agent “remembers” the earlier chat. Swap in a new ID to start a completely fresh conversation.

In [ ]:
config = {
    "configurable": {
        "thread_id": "1"
    }
}

### 🧪 Step 9: Test the Agent

1.	First call – We ask “How many legs do cats have?” The agent chooses to call the tool, gets back the canned fact, and responds.
2.	Second call – With the same thread_id, we ask about dogs. Because the earlier exchange is stored in memory, the agent sees full context and can keep the dialogue flowing naturally.
3.	Printing the agent's reponse


In [ ]:
cat_response = agent.invoke(
    {"messages": [{"role": "user", "content": "How many legs cats have?"}]},
    config
)

In [ ]:
# Continue the conversation using the same thread_id
dog_response = agent.invoke(
    {"messages": [{"role": "user", "content": "What about dogs?"}]},
    config
)

In [ ]:
first_response = cat_response["messages"][-1]
print("Agent:", first_response.content)

In [ ]:
second_response = dog_response["messages"][-1]
print("Agent:", second_response.content)

## What's Next

Click the Jupyter logo in the upper-left corner to open the file tree view and try applying what you've learned in the `Activity.ipynb` notebook.